In [312]:
# https://github.com/ChrizH/pdfstructure

In [15]:
import requests

data = requests.get("https://raw.githubusercontent.com/qiyuyang16/CS4300_microGoogle/master/streamlit_testing/test-pdf.json").json()

In [313]:
query = input("Enter a query")

Enter a queryhi


In [25]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

In [305]:
def text_on_page(dict_var, id_json, list_res, page):
    if type(dict_var) is dict:
        for k, v in dict_var.items():
            if k == id_json and v == page:
                if v > page: return list_res
                list_res.append(dict_var["text"])
            elif isinstance(v, dict):
                text_on_page(v, id_json, list_res, page)   
            elif isinstance(v, list):
                for item in v:
                    text_on_page(item, id_json, list_res, page)
    return list_res

In [360]:
"I\n saw a man in the building\n with a suitcase".split("\n")

['I', ' saw a man in the building', ' with a suitcase']

In [363]:
def get_page(json, page, next_line_split=False):
    lines = []
    for chunk in data["elements"]:
        if next_line_split:
            lines_unsplit = text_on_page(chunk, "page", [], page)
            for l in lines_unsplit:
                more_lines = l.split("\n")
                lines.extend(more_lines)
        else:
            lines.extend(text_on_page(chunk, "page", [], page))             
    return lines

In [386]:
get_page(data, 2, True)


['00doenges-FM  2/2/04  11:54 AM  Page i',
 'Urinary elimination, readiness for enhanced 558–561',
 'Urinary incontinence, functional',
 'Urinary incontinence, reflex 564–566',
 'Urinary incontinence, stress 567–569',
 'Urinary incontinence, urge 572–575',
 'Urinary urge incontinence, risk for 576–578',
 'Incontinence, total',
 'Urinary retention 578–581',
 '570–572',
 '561–564',
 '232–236',
 'ACTIVITY-EXERCISE PATTERN',
 'Activity intolerance, risk for 60–63',
 'Activity intolerance (specify level) 63–65',
 'Adaptive capacity, decreased, intracranial 316–319',
 'Infant behavior, disorganized 295–301',
 'Infant behavior, risk for disorganized 303–304',
 'Infant behavior, readiness for enhanced organized 301–303',
 'Fatigue',
 'Physical mobility, impaired 333–337',
 'Bed mobility, impaired 331–333',
 'Walking, impaired 597–599',
 'Wheelchair mobility, impaired 337–339',
 'Transfer ability, impaired 544–546',
 'Development, risk for delayed 194–197',
 'Autonomic dysreflexia 92–95',
 'Aut

In [327]:
def jaccard(query, text):
    query = strip_non_ascii(query)
    text = strip_non_ascii(text)
    query_set = set(query.split())
    text_set = set(text.split())
    return len(query_set.intersection(text_set)) / len(query_set.union(text_set))

In [324]:
# Returns dict with attributes 'page' and 'score'
def most_similar_page(query):
    top = dict()
    top["page"] = 0
    top["score"] = -1
    
    page = 1
    page_data = get_page(data, page)
    while (len(page_data) > 0):
        page_text = " ".join(page_data)
        jacc_sim = jaccard(query, page_text)
        if jacc_sim > top["score"]:
            top["score"] = jacc_sim
            top["page"] = page
        page += 1
        page_data = get_page(data, page)
    return top


In [350]:
def distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [351]:
distance("alteration of immune system", "3r1p")

26

In [377]:
def max_push(lst, score, elem, maximum):
    lst.append((score, elem))
    lst.sort(key=lambda tup: -1 * tup[0])  
    return lst[0:maximum]

In [382]:
def similar_context(query, page_num):
    sim_list = []
    #min_dist = float('inf')
   
    page_data = get_page(data, page_num, True)
    for phrase in page_data:
        dist = distance(query, phrase)
        sim = jaccard(query, phrase)
        #if min_dist > dist:
          #  min_dist = dist
          #  context = phrase
        sim_list = max_push(sim_list, sim, phrase, 10)
    return sim_list

In [372]:
get_page(data, 55, True)

['03doenges-03  2/2/04  11:57 AM  Page 38',
 'TEACHING/LEARNING—Ability  to  incorporate  and  use',
 'information to achieve healthy lifestyle/optimal wellness',
 'Development, risk for delayed 194–197',
 'Growth and Development, delayed 266–271',
 'Growth, risk for disproportionate 271–275',
 'Health-Seeking Behaviors (specify) 278–281',
 'Knowledge, deficient (specify)',
 'Knowledge (specify), readiness for enhanced 323–325',
 'Noncompliance [Adherence, ineffective] [specify] 343–347',
 'Therapeutic Regimen Management: community, ineffective',
 '319–323',
 'Therapeutic Regimen Management: effective 517–519',
 'Therapeutic Regimen Management: family, ineffective',
 'Therapeutic Regimen Management: ineffective',
 'Therapeutic Regimen Management: readiness for enhanced',
 '522–525',
 '515–517',
 '520–522',
 '525–527',
 'Information that appears in brackets has been added by the authors',
 'to clarify and enhance the use of NDs.',
 'Please  also  see  the  NANDA  diagnoses  grouped  acc

In [388]:
q = "Many years ago, the nursing profession"
top = most_similar_page(q)
print(top)
top_page = top["page"]
for pair in similar_context(q, top_page):
    print(pair)

{'page': 18, 'score': 0.02727272727272727}
(0.6666666666666666, 'Many years ago, the nursing profession identified a problem-')
(0.16666666666666666, 'fying  the  parameters  of nursing  with  the  goal  of attaining')
(0.15384615384615385, 'the  art  of nursing  with  the  most  relevant  elements  of systems')
(0.15384615384615385, 'national acceptance as the basis for providing effective nursing')
(0.15384615384615385, 'Use  of the  nursing  process  requires  the  skills  of (1)  assess-')
(0.14285714285714285, 'nursing  process was  introduced  in  the  1950s  and  has  gained')
(0.14285714285714285, 'Nurses have struggled for years to define nursing by identi-')
(0.13333333333333333, 'ing curricula and is accepted in the legal definition of nursing in')
(0.13333333333333333, 'the  nurse  practice  acts  of most  states. This  nursing  process  is')
(0.08333333333333333, 'are important for the nurse to consider:')


In [331]:
top

{'page': 38, 'score': 0.037037037037037035}